In [1]:
# Test dataset loading

In [1]:
from pathlib import Path

from torch.utils.data import DataLoader

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import lightning as L

import mlflow
import mlflow.pytorch
from mlflow import MlflowClient

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('./datasets')
sys.path.append('./models')

from UNSW_NB15 import UNSWNB15Dataset
from SimpleAE import SimpleAE

In [4]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median').set_output(transform='pandas')),
    ('scaler', StandardScaler().set_output(transform='pandas'))
])

In [5]:
data_dir = Path("../data/UNSW-NB15/preprocessed")
train_dataset = UNSWNB15Dataset(data_dir, num_records=100000, normal=0.999, transformer=pipeline)
test_dataset = UNSWNB15Dataset(data_dir, num_records=10000, normal=0.5, test_for=train_dataset, transformer=pipeline)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=65536, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

In [ ]:
# for x, y in train_loader:
#     print(x)
#     print(y)
#     break

In [7]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment("Config test")

mlflow.pytorch.autolog()


2025/03/31 23:02:33 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


In [8]:


input_size = train_dataset[0][0].shape[0]
print("Input dim:", input_size)

model = SimpleAE(input_size, 16)

trainer = L.Trainer(accelerator='gpu', max_epochs=200)

with mlflow.start_run() as run:
    trainer.fit(model, train_loader, test_loader)

Input dim: 204


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
2025/03/31 23:02:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Szymon\AppData\Roaming\Python\Python311\site-packages\mlflow\pytorch\_lightning_autolog.py:463: UserWarning: Autologging is known to be compatible with pytorch-lightning versions between 1.9.0 and 2.2.4 and may not succeed with packages outside this range."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 476 K  | train
1 | decoder | Sequential | 476 K  | train
-----------------------------------------------
953 K     Trainable params
0         Non-trainable params
953 K     Total params
3.812     Total estimated model params 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\Szymon\AppData\Roaming\Python\Python311\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\Szymon\AppData\Roaming\Python\Python311\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\Szymon\AppData\Roaming\Python\Python311\site-packages\lightning\pytorch\loops\fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=150` reached.
2025/03/31 23:07:33 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [9]:
print("threshold:", model.threshold)

trainer.validate(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2734e8a0838547ada1efb9ebbf0c382a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow


threshold: tensor(0.7191, device='cuda:0', grad_fn=<AddBackward0>)


2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bd9db8526db745edbed8d58b2ba2f02a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8564a504946e44e0b5330d405c41cac9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '03e03fccc3264f1f98ab2ea5d3dc56ce', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow


Validation: |          | 0/? [00:00<?, ?it/s]

2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bbe20b99971c4e68af89f2e94c14961d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a2912d5cc1474659aa3e7b442c022961', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'dc0eb92e12064b47baa37f88f67cd23e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
2025/03/31 23:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '63e4de9f5beb47aa8c445d88a475df11', which will track hyperparameters, performance metrics, model artifacts, and lineage i

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       positive_rate       │   0.008700000122189522    │
│       val_accuracy        │            0.5            │
│         val_loss          │    -5.820711612701416     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': -5.820711612701416,
  'val_accuracy': 0.5,
  'positive_rate': 0.008700000122189522}]